In [84]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import urllib.request
import tensorflow as tf 

In [85]:
weight_urls = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weight_file = "inception_v3.h5"
urllib.request.urlretrieve(weight_urls, weight_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x181455fa330>)

In [86]:
#create an InceptionV3 instance 
pretrain_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = None)
pretrain_model.load_weights(weight_file)

In [87]:
# pretrain_model.summary()
#It's too huge and long-listed that i need to put it in "#"

In [88]:
for layer in pretrain_model.layers: 
    layer.trainable = False


last_layer = pretrain_model.get_layer('mixed7')
last_output = last_layer.output
print("last layer output shape: ", last_output.shape)

last layer output shape:  (None, 7, 7, 768)


In this step, we freeze the entire network from retraining and set a variable to "mixed7" as where we want to crop the entire network up to. We're free to experiment with other layers (mixed5,...)

In [89]:
import tensorflow as tf 
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, Flatten

#Flatten the output layer to 1 dim
x = layers.Flatten()(last_output)
#Adding dense layer 
x = layers.Dense(1024, activation = 'relu')(x)
#Adding output layer 
x = layers.Dense(1, activation = 'sigmoid')(x)

explain: <code>layers.Flatten()</code> create an instance of Flatten layer. <code>(last_output)</code> then applied that layer to Last_output. Its the same as writing <br> <li><code>flatten_layer = layers.Flatten()</code>  # Create a Flatten layer</li> <br>
<li><code>x = flatten_layer(last_output)</code>  # Apply it to last_output</li>


In [90]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop 
#define model
model = Model(pretrain_model.input, x)
#Logic: các tham số trong Model() bao gồm input từ pretrain_model, output từ last_layer.output được nối với tiếp với các lớp X mà ta đã định nghĩa

In [91]:
#Checking on the new model 
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_188 (Conv2D) │ (None, 74, 74,    │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 74, 74,    │         96 │ conv2d_188[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_188      │ (None, 74, 74,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_189 (Conv2D) │ (None, 72, 72,    │      9,216 │ activation_188[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 72, 72,    │         96 │ conv2d_189[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_189      │ (None, 72, 72,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_190 (Conv2D) │ (None, 72, 72,    │     18,432 │ activation_189[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 72, 72,    │        192 │ conv2d_190[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_190      │ (None, 72, 72,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 35, 35,    │          0 │ activation_190[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_191 (Conv2D) │ (None, 35, 35,    │      5,120 │ max_pooling2d_8[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │        240 │ conv2d_191[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_191      │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_192 (Conv2D) │ (None, 33, 33,    │    138,240 │ activation_191[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 33, 33,    │        576 │ conv2d_192[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_192      │ (None, 33, 33,    │          0 │ batch_normalizat

 Total params: 47,512,481 (181.25 MB)

 Trainable params: 38,537,217 (147.01 MB)

 Non-trainable params: 8,975,264 (34.24 MB)

In [92]:
model.compile(optimizer = RMSprop(learning_rate= 0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [93]:
training_dir = r"D:\Machine learning\Lập trình\Learning ML and AI for Coder\Chapter 3\dataset\horse-or-human\training"
testing_dir = r"D:\Machine learning\Lập trình\Learning ML and AI for Coder\Chapter 3\dataset\horse-or-human\validation"

In [94]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [95]:
#Create an instancec of ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1/255, 
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True, 
    fill_mode = 'nearest'
)
image_generator = train_datagen.flow_from_directory(
    training_dir, 
    target_size = (150, 150),
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [96]:
history = model.fit(
    image_generator, 
    epochs = 10
)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.8590 - loss: 0.3311
Epoch 2/10
 5/33 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9697 - loss: 0.0571

KeyboardInterrupt: 